In [1]:
from sklearn.datasets import load_iris

In [2]:
iris= load_iris()

In [3]:
X=iris['data'][:,(2,3)]
y=iris['target']

In [7]:
import numpy as np
X_new = np.c_[np.ones([len(X), 1]), X]

In [10]:
from sklearn.model_selection import train_test_split

In [12]:
total_size = len(X)
rnd_indices = np.random.permutation(total_size )

In [28]:
test_size = int(total_size * 0.2)
val_size = int(total_size * 0.2)
train_size = int(total_size * 0.6)

In [29]:
X_train = X_new[rnd_indices[:train_size]]
X_val = X_new[rnd_indices[train_size:-test_size] ]
X_test = X_new[rnd_indices[test_size:]]

In [32]:
y_train = y[rnd_indices[:train_size]]
y_val = y[rnd_indices[train_size:-test_size] ]
y_test = y[rnd_indices[test_size:]]

$\sigma\left(\mathbf{s}(\mathbf{x})\right)_k = \dfrac{\exp\left(s_k(\mathbf{x})\right)}{\sum\limits_{j=1}^{K}{\exp\left(s_j(\mathbf{x})\right)}}$

In [38]:
def softmax(logits):
    exps = np.exp(logits)
    exp_sums = np.sum(exps, axis=1, keepdims=True)
    return exps / exp_sums

In [39]:
def to_one_hot_encode(y):
    m=len(y)
    n_classes=y.max() + 1
    y_one_hot = np.zeros((m,n_classes))
    y_one_hot[np.arange(m),y] = 1
    return y_one_hot

In [42]:
y_test[:5]

array([0, 2, 0, 1, 0])

In [43]:
to_one_hot_encode(y_test[:5])

array([[1., 0., 0.],
       [0., 0., 1.],
       [1., 0., 0.],
       [0., 1., 0.],
       [1., 0., 0.]])

In [44]:
y_train = to_one_hot_encode(y_train)
y_val = to_one_hot_encode(y_val)
y_test = to_one_hot_encode(y_test)

In [45]:
n_inputs = X_train.shape[1]
n_outputs=y_train.shape[1]

In [46]:
n_inputs,n_outputs

(3, 3)


Now here comes the hardest part: training! Theoretically, it's simple: it's just a matter of translating the math equations into Python code. But in practice, it can be quite tricky: in particular, it's easy to mix up the order of the terms, or the indices. You can even end up with code that looks like it's working but is actually not computing exactly the right thing. When unsure, you should write down the shape of each term in the equation and make sure the corresponding terms in your code match closely. It can also help to evaluate each term independently and print them out. The good news it that you won't have to do this everyday, since all this is well implemented by Scikit-Learn, but it will help you understand what's going on under the hood.

So the equations we will need are the cost function:

$J(\mathbf{\Theta}) = \dfrac{1}{m} \sum\limits{i=1}^{m} { \sum\limits{k=1}^{K}{y_k^{(i)}\log\left(\hat{p}_k^{(i)}\right)} }$
And the equation for the gradients:

$\nabla_{\mathbf{\theta}^{(k)}} \, J(\mathbf{\Theta}) = \dfrac{1}{m} \sum\limits_{i=1}^{m}{ \left ( \hat{p}^{(i)}_k - y_k^{(i)} \right ) \mathbf{x}^{(i)}}$

Note that $\log\left(\hat{p}_k^{(i)}\right)$ may not be computable if $\hat{p}_k^{(i)} = 0$. So we will add a tiny value $\epsilon$ to $\log\left(\hat{p}_k^{(i)}\right)$ to avoid getting nan values.

In [53]:
eta = 0.01
epochs=5001
theta = np.random.randn(n_inputs,n_outputs)
epsilon=1e-7
m =len(X_train)
for epoch in range(epochs):
    
    soft_max_scores = X_train.dot(theta)
    Y_proba = softmax(soft_max_scores)
    loss = -np.mean( np.sum(y_train * np.log(Y_proba + epsilon ) , axis =1 ) )
    error = Y_proba - y_train
    if epoch % 500 == 0:
        print(epoch, loss)
    grad = 1/m * X_train.T.dot(error)
    theta = theta - eta *grad
    

0 2.225258233666248
500 0.7437784012968843
1000 0.6276448772950922
1500 0.5579846694734841
2000 0.511893151682225
2500 0.4788211463479246
3000 0.45359027596758156
3500 0.4334394978980776
4000 0.4167805541015985
4500 0.4026389227094392
5000 0.3903836007492423


In [54]:
theta

array([[ 3.3655896 , -0.59552686, -2.51758943],
       [-1.73073134, -0.60592296, -0.97419478],
       [-1.54670082, -0.34553142,  2.01757374]])

In [56]:
val_soft_max = X_val.dot(theta)
y_val_prob = softmax(val_soft_max)

In [67]:
y_val_acutal = np.argmax(y_val, axis = 1)
y_val_pred = np.argmax(y_val_prob, axis = 1)

In [68]:
np.mean( y_val_acutal == y_val_pred)

0.9666666666666667

In [118]:
eta = .01
epochs=10001
theta = np.random.randn(n_inputs,n_outputs)
epsilon=1e-7
alpha = 0.1
m =len(X_train)
for epoch in range(epochs):
    
    soft_max_scores = X_train.dot(theta)
    Y_proba = softmax(soft_max_scores)
    loss = -np.mean( np.sum(y_train * np.log(Y_proba + epsilon ) , axis =1 ) )
    l2_loss = 1/2 *np.sum(np.square(theta[1:]))
    loss += (alpha *l2_loss)
    error = Y_proba - y_train
    if epoch % 500 == 0:
        print(epoch, loss)
    grad = 1/m * X_train.T.dot(error) + np.r_[np.zeros([1, n_outputs]), alpha * theta[1:]]
    theta = theta - eta *grad
    

0 5.470138484888197
500 0.8604052467241752
1000 0.6818280052802425
1500 0.6132498830555185
2000 0.5814886286157532
2500 0.5638263785048387
3000 0.5524943944883337
3500 0.5444807617886854
4000 0.5384459499436459
4500 0.5337085681619916
5000 0.5298799051654057
5500 0.5267179999380633
6000 0.5240624898599257
6500 0.5218021550305435
7000 0.5198571433291109
7500 0.5181684909872433
8000 0.516691601215912
8500 0.515392022718504
9000 0.5142426339863428
9500 0.5132217192906859
10000 0.5123116263470308


In [119]:
y_val_soft_max = X_val.dot(theta)
y_val_pred = softmax(y_val_soft_max)

y_val_pred = np.argmax(y_val_pred, axis = 1)
y_val_act = np.argmax(y_val,axis = 1)

In [120]:
np.mean(y_val_pred==y_val_act)

1.0

In [93]:
theta

array([[ 2.80126265, -0.9263831 , -2.94646634],
       [-0.85291029,  0.30455043,  0.53971364],
       [-0.40556752, -0.11221168,  0.51504657]])

In [123]:
y_test_soft_max = X_test.dot(theta)
y_test_pred = softmax(y_test_soft_max)

y_test_pred = np.argmax(y_test_pred, axis = 1)
y_test_act = np.argmax(y_test,axis = 1)

In [124]:
np.mean(y_test_pred==y_test_act)

0.9583333333333334

In [125]:
len(y_test)

120

In [126]:
np.sum(y_test_pred==y_test_act)

115

In [132]:
len(X_test)

120